In [1]:
!pip3 install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Uninstalling tenacity-8.0.1:
      Successfully uninstalled tenacity-8.

In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model="llama2")
llm.invoke("The first man on the moon was ...")

'\nThe first man on the moon was Neil Armstrong. He stepped onto the moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first person to walk on the lunar surface.'

In [4]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'\nCertainly! Here are the categories for each of the expenses you provided:\n\n* Spotify AB by Adyen - Entertainment\n* Beta Boulders Ams Amsterdam Nld - Sports\n* Taxi Utrecht - Transportation\n* Ministerie van Justitie en Veiligheid - Government\n* Etos AMSTERDAM NLD - Grocery\n* Bistro Bar Amsterdam - Food and Drink\n\nI hope this helps! Let me know if you have any other questions.'

### Read transaction data

In [5]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63


In [6]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

23

In [7]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

### Categorise bank transactions with Llama2

In [8]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 23]

In [9]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [10]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

['', 'Of course! Here are the appropriate categories for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. ISS Catering Services De Meern - Food & Beverage', '3. Taxi Utrecht - Transportation', '4. Etos AMSTERDAM NLD - Grocery', '5. Bistro Bar Amsterdam - Dining']


,Transaction vs category,Transaction,Category
0,,,None
1,Of course! Here are the appropriate categories...,Of course! Here are the appropriate categories...,None
2,,,None
3,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
4,2. ISS Catering Services De Meern - Food & Bev...,2. ISS Catering Services De Meern,Food & Beverage
5,3. Taxi Utrecht - Transportation,3. Taxi Utrecht,Transportation
6,4. Etos AMSTERDAM NLD - Grocery,4. Etos AMSTERDAM NLD,Grocery
7,5. Bistro Bar Amsterdam - Dining,5. Bistro Bar Amsterdam,Dining


In [11]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Sure! Here are the categories I came up with for each expense:', '', '1. Spotify AB by Adyen - Entertainment', '2. Belastingdienst - Utilities', '3. Tesco Breda - Groceries', '4. Monthly Appartment Rent - Housing', '5. Vishandel Sier Amsterdam - Shopping', '6. Selling Paintings - Art/Hobbies', '7. Spotify AB By Adyen - Music Streaming', '8. Tk Maxx Amsterdam DA - Clothing/Shoes', '9. Consulting - Professional Services', '10. Aidsfonds - Donations/Charity', '11. TLS BV Inz Ov-Chipkaart - Transportation', '12. Etos Amsterdam - Health/Beauty', '13. Beta Boulders Ams Amsterdam - Fitness/Sports', '14. Salary - Personal Services', '15. Bouldermuur BV Amsterdam - Personal Services', '16. Birtat Restaurant Amsterdam - Food/Dining', '17. Freelancing - Professional Services', '18. Tikkie - Financial Services', '19. Blogging - Writing/Journalism', '20. Taxi Utrecht - Transportation', '21. Apple Services - Technology', '22. Amazon Lux - Shopping', '23. Classpass* Monthly - Fitness/Sports', '24. 

In [12]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Sure! Here are the categories I came up with f...,Sure! Here are the categories I came up with f...,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Belastingdienst - Utilities,2. Belastingdienst,Utilities
4,3. Tesco Breda - Groceries,3. Tesco Breda,Groceries
5,4. Monthly Appartment Rent - Housing,4. Monthly Appartment Rent,Housing
6,5. Vishandel Sier Amsterdam - Shopping,5. Vishandel Sier Amsterdam,Shopping
7,6. Selling Paintings - Art/Hobbies,6. Selling Paintings,Art/Hobbies
8,7. Spotify AB By Adyen - Music Streaming,7. Spotify AB By Adyen,Music Streaming
9,8. Tk Maxx Amsterdam DA - Clothing/Shoes,8. Tk Maxx Amsterdam DA,Clothing/Shoes


In [13]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [14]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Entertainment', 'Utilities', 'Groceries', 'Housing',
       'Shopping', 'Art/Hobbies', 'Music Streaming', 'Clothing/Shoes',
       'Professional Services', 'Donations/Charity', 'Transportation',
       'Health/Beauty', 'Fitness/Sports', 'Personal Services',
       'Food/Dining', 'Financial Services', 'Writing/Journalism',
       'Technology'], dtype=object)

In [15]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

/var/folders/p2/05zzb9jx10x6f3chs84fjng80000gn/T/ipykernel_26341/1238705727.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
/var/folders/p2/05zzb9jx10x6f3chs84fjng80000gn/T/ipykernel_26341/1238705727.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
/var/folders/p2/05zzb9jx10x6f3chs84fjng80000gn/T/ipykernel_26341/1238705727.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [16]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

/var/folders/p2/05zzb9jx10x6f3chs84fjng80000gn/T/ipykernel_26341/2440992137.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
/var/folders/p2/05zzb9jx10x6f3chs84fjng80000gn/T/ipykernel_26341/2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,Spotify AB by Adyen,Entertainment
3,2. Belastingdienst - Utilities,Belastingdienst,Utilities
4,3. Tesco Breda - Groceries,Tesco Breda,Groceries
5,4. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
6,5. Vishandel Sier Amsterdam - Shopping,Vishandel Sier Amsterdam,Shopping
7,6. Selling Paintings - Art/Hobbies,Selling Paintings,Art/Hobbies
8,7. Spotify AB By Adyen - Music Streaming,Spotify AB By Adyen,Music Streaming
9,8. Tk Maxx Amsterdam DA - Clothing/Shoes,Tk Maxx Amsterdam DA,Clothing
10,9. Consulting - Professional Services,Consulting,Services
11,10. Aidsfonds - Donations/Charity,Aidsfonds,Donations/Charity


In [17]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,2. Belastingdienst - Utilities,Belastingdienst,Utilities
1,2023-12-30,Tesco Breda,Expense,17.53,3. Tesco Breda - Groceries,Tesco Breda,Groceries
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,4. Monthly Appartment Rent - Housing,Monthly Appartment Rent,Housing
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,5. Vishandel Sier Amsterdam - Shopping,Vishandel Sier Amsterdam,Shopping
4,2023-12-29,Selling Paintings,Income,13.63,6. Selling Paintings - Art/Hobbies,Selling Paintings,Art/Hobbies
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,NaN,NaN,NaN
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,NaN,NaN
7,2023-12-22,Consulting,Income,541.57,9. Consulting - Professional Services,Consulting,Services
8,2023-12-22,Aidsfonds,Expense,10.70,10. Aidsfonds - Donations/Charity,Aidsfonds,Donations/Charity
9,2023-12-20,Consulting,Income,2641.93,9. Consulting - Professional Services,Consulting,Services


In [18]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)